<a href="https://colab.research.google.com/github/Chirag314/LLM/blob/main/LLM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UNDERSTANDING RETRIEVAL QUESTION ANSWERING

In [ ]:
!pip install -Uqqq rich openai tiktoken wandb tenacity langchain unstructured tabulate pdf2image chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tiktoken
import wandb
from pprint import pprint
from wandb.integration.openai import autolog
import random

In [ ]:
import os,random
from getpass import getpass
import openai
from pathlib import Path
from rich.markdown import Markdown
import pandas as pd
from tenacity import(
    retry,
    stop_after_attempt,
    wait_random_exponential,
)


In [ ]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['COLAB' in x for x in os.environ.keys()]):
    print('Please enter password in the prompt at the top of your window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

Please enter password in the prompt at the top of your window!
Paste your OpenAI key from: https://platform.openai.com/account/api-keys
··········


Langchain
Langchain is a framework for developing applications powered by language models. LEts use it in the code.

In [ ]:
#trace langchain with W&B
os.environ['LANGCHAIN_WANDB_TRACING']="true"

os.environ["WANDB_PROJECT"]="llmapps"

Parsing Documents: We will use a small sample of markdown documents in this notebook.

In [ ]:
model_name="text-davinci-003"

In [ ]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

Cloning into 'edu'...
remote: Enumerating objects: 2493, done.
remote: Counting objects: 100% (1036/1036), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 2493 (delta 723), reused 858 (delta 642), pack-reused 1457
Receiving objects: 100% (2493/2493), 22.60 MiB | 14.31 MiB/s, done.
Resolving deltas: 100% (1419/1419), done.


In [ ]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
  "Find all markdown files in a directory and return a LangChain document"
  dl=DirectoryLoader(directory, "**/*.md")
  return dl.load()

documents=find_md_files('../docs_sample/')
len(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


11

In [ ]:
# We will need to count tokens in the documents and for that we need the tokenizer
tokenizer=tiktoken.encoding_for_model(model_name)

In [ ]:
#Function to count number of tokens
def count_tokens(documents):
  token_counts=[len(tokenizer.encode(document.page_content)) for document in documents]
  return token_counts

count_tokens(documents)

[763, 310, 1154, 2135, 2047, 2616, 665, 387, 2592, 2330, 1199]

We will use LangChain built in MarkdownTextSplitter to split the documents into sections. Actually splitting Markdown without breaking syntax is not that easy. This splitter strips out syntax.
1. We can pass the chunk_size param and avoid lengthy chunks.
2. The chunk_overlap param is useful so you dont cut sentences randomly. This is less necessary with Markdown



In [ ]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter=MarkdownTextSplitter(chunk_size=1000)
document_sections=md_text_splitter.split_documents(documents)
len(document_sections),max(count_tokens(document_sections))

(90, 438)

In [ ]:
Markdown(document_sections[0].page_content)

import Tabs from '@theme/Tabs'; import TabItem from '@theme/TabItem';                                              

Tags                                                                                                               

Tags can be used to label runs with particular features that might not be obvious from the logged metrics or       
Artifact data -- this run's model is in_production, that run is preemptible, this run represents the baseline.     

How to add tags                                                                                                    

You can add tags to a run when it is created: wandb.init(tags=["tag1", "tag2"]) .                                  

You can also update the tags of a run during training (e.g. if a particular metrics crosses a pre-defined          
threshold):                                                                                                        

                                                                                                                   
 run = wandb.init(entity="entity", project="capsules", tags=["debug"])                                             
                                                                                                                   
 ...                                                                                                               
                                                                                                                   
 if current_loss < threshold:                                                                                      
     run.tags = run.tags + ("release_candidate",)                                                                  
                                                                                                                   


Embeddings
Lets now use embedding with a vector ddatabase retriever to find relevant documents for a query

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

#openai to embed text and chroma to store the vectors
embeddings=OpenAIEmbeddings()
db=Chroma.from_documents(document_sections,embeddings)

We can create a retriever from the db now , we can pass the k param to get the most relevaant sectons from the similarity search

In [ ]:
retriever=db.as_retriever(search_kwargs=dict(k=3))

In [17]:
query="How can I share my W&B report with my team members in a public W&B project?"
docs=retriever.get_relevant_documents(query)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Streaming LangChain activity to W&B at https://wandb.ai/cdesai/llmapps/runs/mpkx96ka
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [18]:
#lets see the results
for doc in docs:
  print(doc.metadata['source'])

../docs_sample/collaborate-on-reports.md
../docs_sample/collaborate-on-reports.md
../docs_sample/teams.md


Suff prompt
Take the content of retrieved documents, stuff them into prompt template along with the query and pass into an LLM to obtain the answer.

In [20]:
from langchain.prompts import PromptTemplate
prompt_template="""Use the following pieces of context to answer the questiona the end.
If you dont know the answer , just say that you dont know, dont try to make up an answer.

{context}

Questin: {question}

Helpful Answer:"""
PROMPT=PromptTemplate(
    template=prompt_template,input_variables=['context','question']
)

context="\n\n".join([doc.page_content for doc in docs])
prompt=PROMPT.format(context=context,question=query)

In [21]:
#use langchain to call openai chat api with the question
from langchain.llms import OpenAI

llm=OpenAI()
response=llm.predict(prompt)
Markdown(response)

To share a report in a public W&B project, select the Share button on the upper right hand corner. You can either  
provide an email account or copy the magic link. Users invited by email will need to log into Weights & Biases to  
view the report. Users who are given a magic link to not need to log into Weights & Biases to view the report.

Using Langchain
Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using RetrievalQA chain.

In [22]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(llm=OpenAI(),chain_type='stuff',retriever=retriever)
result=qa.run(query)
Markdown(result)

To share a report, select the Share button on the upper right hand corner. You can either provide an email account 
or copy the magic link. Users invited by email will need to log into Weights & Biases to view the report. Users who
are given a magic link to not need to log into Weights & Biases to view the report. Shared reports are view-only.

In [23]:
import wandb
wandb.finish()